In [214]:
import pandas as pd
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense
from tensorflow.keras.utils import to_categorical
import numpy as np

In [2]:
train_df=pd.read_csv('train.csv')
train_df.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [3]:
texts=train_df['text'].values

In [11]:
stops=stopwords.words('english')

In [19]:
stops=list(string.punctuation)+stops

In [20]:
words=word_tokenize(texts[0])
words

['@',
 'SouthwestAir',
 'I',
 'am',
 'scheduled',
 'for',
 'the',
 'morning',
 ',',
 '2',
 'days',
 'after',
 'the',
 'fact',
 ',',
 'yes..not',
 'sure',
 'why',
 'my',
 'evening',
 'flight',
 'was',
 'the',
 'only',
 'one',
 'Cancelled',
 'Flightled']

In [26]:
tag=pos_tag([words[1]])[0][1]
tag

'NN'

In [71]:
def get_simple_pos_tag(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    if tag.startswith('V'):
        return wordnet.VERB
    if tag.startswith('R'):
        return wordnet.ADV
    return wordnet.NOUN

In [67]:
get_simple_pos_tag(tag)

'n'

In [68]:
lemmatizer=WordNetLemmatizer()

In [69]:
lemmatizer.lemmatize(words[1],get_simple_pos_tag(tag))

'SouthwestAir'

In [75]:
def clean_document(document):
    cleaned_document=[]
    words=word_tokenize(document)
    for w in words:
        w=w.lower()
        if (not w in stops) and w.isalpha():
            root_w=lemmatizer.lemmatize(w,get_simple_pos_tag(pos_tag([w])[0][1]))
            cleaned_document.append(root_w)
    return ' '.join(cleaned_document)     

In [86]:
def get_cleaned_texts(texts):
    cleaned_texts=[]
    for text in texts:
        cleaned_text=clean_document(text)
        print(cleaned_text)
        cleaned_texts.append(cleaned_text)
    return cleaned_texts    

In [87]:
cleaned_texts=get_cleaned_texts(texts)

southwestair schedule morning day fact sure even flight one cancel flightled
southwestair see worker time time go beyond love fly guy thank
united flew ord miami back great crew service leg thanks
southwestair horse radish
united flight ord delayed air force one last flight sbn min land
united load u fly sardine knew pilot hour late flight incompetent beyond belief
jetblue stock response delay frustrate poor cust serv amp told ppl wait amp come back
jetblue nice hop rack enough mile take trip seattle enjoy perfect latte city coffee
united frankly bad customer service ever problem happen deal defines company never united
southwestair yeah haha never one expensive much fun destinationdragons
southwestair gt dca flight almost full people screw cancel flightation united usairways cancel flight
jetblue easy way get ticket receipt get one check get one online thanks
usairways love change lounge cheese veggie olive addition cracker snack mix
southwestair receive bad customer service end spend

united quick serious question show quality service well place http
united get first flight day ground time let go
southwestair southwest mobile boarding pass available io passbook http http
southwestair stuck fort lauderdale
jetblue clear folder regularly jetblue white list never get
united keep flight cheap time good flew americanair cancel flightled flight time last month united gt
jetblue believe irina super disappointed eatup cafe available bos gt sfo flight today
usairways flight place original flight charlotte cancel flightled delay
usairways captain turn seat belt sign people get walk around good
united http flight go get plane businesstravel goodenoughmother
americanair call chairman call emerald today call former customer
jetblue saw one useless tsa agent inspect shoe
united thank response
americanair one receive text alert automate call anything either literally information happen
usairways make send bloody mary row
united gate change two delay even crew think airline suck
us

united need apology need bag take minute land get unacceptable
americanair see photo scale model amp pn wheel deflator specify chapter amm http
united fly stl saturday expect ice amp snow start look travel waiver thanks
americanair thanks get back find airline future
virginamerica know need spotify stat guiltypleasures
united patient luxurious middle seat next delayed flight
usairways hour tarmac charleston still ca get response week unacceptable
southwestair happen customer service cancel flightled flight day amp uninformed unapologetic staff nolove cali
united sfo hnl zero entertainment system youareonyourown
americanair overweight pull dozen passenger luggage seriously
jetblue disappointed flight back boston pr cancel flightled rebooked next day delayed one compensation
united well customer service john wayne airport
americanair wait delayed plane sat plane hour wait late flight passenger b
united always help communicate customer
united customer service unbelievably bad abysmal phon

southwestair right delayed crabby also allow one family member purchase earlybird check hold row
united guy suck
united thank person houston could get flight rout newark nj go sfo
united lose bag huston leave bahamas cruise today nola buy new stuff customer service non existent
southwestair thx customer service rep alex patient help reflight book problem cancel flight flight get u tmrw
usairways faster way get columbia sc make stay clt night delayed flight
americanair highly disappointed baggage service merge usairways guy lose bag miss item
southwestair program complete joke company value loyal look another airliner
americanair understand need drain lavatory fear freeze policy check boarding smh come
united hope nicer flight time try thing right sell ticket connetion
southwestair thank thank thank last attempt win destinationdragons ticket success could happier
jetblue hate http
americanair cvgairport get together perfect combination delay cancel flightlations
united may dislike delay

southwestair really unhelpful ca rebook online http
jetblue flight full
usairways rock hit plane delay never easy day u airway ya never fail let
jetblue thank excite fly first time
united gate definitely full park wheel chocked customer service call unmet arrival
southwestair provide direct assistance colleague ticket book via corporate account help
aww jetblue guess need book flight somewhere warm asap suggestion
united never fly guy ever customer service piss poor best
yes thanks much united whitterbug see spoke reservation team reinstate flight thanks
usairways americanair anyone answer phone today listen damn message minute
southwestair thank
americanair beyond redemption jfk baggage claim look like luggage warehouse
united know two hour u sit plane bag load plane weight
americanair deplane fuck time fuck aa
united joni great job flight lax thanks great flight
americanair willing hold agent helpful keep hang americanairsucks hiremorepoeple
united delayed another hour
usairways admi

...

southwestair would eternally grateful ticket vega show god bless
jetblue wife book last minute flight funeral separately would like seat dm conf
southwestair flight sunny sfo delayed hour others cancel flightled left right little update help southwestair
southwestair expect customer service dallas available
americanair surprised fast respond u brand twitter also fast world
southwestair fan imagine dragon since fave band destinationdragons bday get tix
united amaze hard talk customer service
united reflight book problem would make reach time meeting also wan na take chance book etihad instead
southwestair start daily flight columbus oh oakland avgeek
jetblue flight delayed hope dont miss connect please fuck girlfriend
usairways lazy inconsiderate unprofessional airline ever travel
united assume would people let united bad plane crew ca fly
united round
usairways need contact asap furious
jetblue cinnabon two team serve cinnabons flight bonsinthesky
americanair answer phone try change re

united good call tomorrow thanks
americanair hey happen zurich jfk appear squawk land london heathrow
united get chance today fly let see redeem last horrendous trip state
united stuck plane hour due mechanical fault update please happen way newark lhr
united wife phone several time past day result
virginamerica ladygaga carrieunderwood sorry mary martin first
americanair need speak real person flight cancel flightled explanation hold hour
americanair dmed
united miss flight flight held minute would make watch pull away gate
southwestair wife group prev flight get bumped reason alone two kid least put group
united tell number call left airport negligent pilot staff
united one operate ramp let u plane iah wow
virginamerica getaway deal may lot cool city http cheapflights farecompare
united wth honest customer well last change drive home plane left
southwestair hey follow u dm question
united well bar set low guy really good apology game change u guy learn
southwestair fly jax den paid w

united thank wonderful global first class flight fra great service cont http
usairways even go acknowledge bumped flight weather arizona
southwestair entire flight luggage miss dca hour nobody know anything
southwestair checkin lame business really last seat broad shoulder haha
united terrible
jetblue dont get wrong love fly tv free bag descent fare recent plane problem annoy
americanair worry blame weather
united frontier http
southwestair nothing express express bag drop phoenix sky harbor
usairways cancel flightlations delay cause miss connect flight another night home thanks pathetic
americanair line sfo customer service rude disorganize congest dissapointing
usairways take day respond already open case
americanair thank contact american email address write unmonitored account
usairways cancel flightled make u wait hour fly usair every week upgrade status proactively
usairways u nope get hour plane pilot announcement one water run planet small water bottle
southwestair accord fligh

united go website flight book problem three ticket today boot system umpteen time
americanair gate agent jan l phoenix least able get young daughter seat near wife thank
americanair yes rno departure dfw lga lax jfk
americanair know reservation team hr callback time
united wo help do
southwestair thanks remind upcoming trip florida really need http
americanair logo look like glass
jetblue someone screen play flight http
americanair nice people ever flight david deane amp norma sedholm make feel comfortable
united scvpools please call
united please tell go make connect flight stl southbendinwhere http
united bad experience ever fly
americanair cancel flight flight today weather dallas without charge
southwestair rescue sundown chicago next stop san antonio http
americanair arrive eastern deplane crjs airstairs icy tarmac valet bag sent carousel
southwestair keep say mobile boarding pass unavailable despite checked everything
united personal device entertainment system great app crash ev

usairways call window seat http
jetblue flight leave boston seattle right flight
usairways reminder tweet americanair join u http
usairways americanair also message try sell valentine day gift week late flight sound right usair
southwestair thank take good care people al http alstdi
united premier gold desk change flight waif fee give wrong flight jana acosta salt lake refuse service angry
united guy name otis ord know customerservice able get bag upgraded sure
united glad mean something lol
usairways get receipt change fee originally book mile
usairways americanair help regard flight kphl would much appreciate
americanair westjet rep wo anything lose never even checked westjet lose
united mind get next flight
southwestair sure second
jetblue jua jfk ask desk incredibly helpful thanks
usairways flight cancel flightled twice minute automate question phone go busy signal help
jetblue seat guaranteed pay call rep say people book seat
jetblue word staff training
oma mile destination spotty

In [88]:
t=Tokenizer()
t.fit_on_texts(cleaned_texts)

In [131]:
t.word_index

{'flight': 1,
 'united': 2,
 'usairways': 3,
 'americanair': 4,
 'southwestair': 5,
 'jetblue': 6,
 'get': 7,
 'http': 8,
 'hour': 9,
 'thanks': 10,
 'cancel': 11,
 'u': 12,
 'service': 13,
 'help': 14,
 'customer': 15,
 'time': 16,
 'call': 17,
 'bag': 18,
 'wait': 19,
 'go': 20,
 'plane': 21,
 'make': 22,
 'fly': 23,
 'need': 24,
 'amp': 25,
 'hold': 26,
 'thank': 27,
 'would': 28,
 'still': 29,
 'one': 30,
 'try': 31,
 'airline': 32,
 'gate': 33,
 'day': 34,
 'please': 35,
 'back': 36,
 'ca': 37,
 'take': 38,
 'delayed': 39,
 'book': 40,
 'say': 41,
 'virginamerica': 42,
 'flightled': 43,
 'bad': 44,
 'seat': 45,
 'delay': 46,
 'like': 47,
 'change': 48,
 'phone': 49,
 'agent': 50,
 'late': 51,
 'miss': 52,
 'today': 53,
 'guy': 54,
 'know': 55,
 'ticket': 56,
 'work': 57,
 'airport': 58,
 'could': 59,
 'give': 60,
 'minute': 61,
 'way': 62,
 'great': 63,
 'lose': 64,
 'well': 65,
 'check': 66,
 'never': 67,
 'travel': 68,
 'even': 69,
 'min': 70,
 'see': 71,
 'problem': 72,
 'want'

In [163]:
vocab_size=len(t.word_index.keys())
vocab_size

8140

In [134]:
x_train=t.texts_to_sequences(cleaned_texts)

In [140]:
def get_max_len(x):
    max_len=0
    for i in x:
        if len(i)>max_len:
            max_len=len(i)
    return max_len        

In [141]:
max_len=get_max_len(x_train)
max_len

22

In [142]:
x_train=pad_sequences(x_train,max_len,padding='pre')

In [143]:
x_train.shape

(10980, 22)

In [158]:
y_train=train_df['airline_sentiment'].values
y_train

array(['negative', 'positive', 'positive', ..., 'negative', 'negative',
       'negative'], dtype=object)

In [159]:
le=LabelEncoder()
y_train=le.fit_transform(y_train)
y_train.shape

(10980,)

In [160]:
y_train=to_categorical(y_train)
y_train.shape

(10980, 3)

In [315]:
model=Sequential()
model.add(Embedding(vocab_size+1,10,input_length=22))

In [316]:
model.predict(x_train).shape

(10980, 22, 10)

In [317]:
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(3,activation='sigmoid'))

In [318]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 22, 10)            81410     
_________________________________________________________________
lstm_21 (LSTM)               (None, 22, 100)           44400     
_________________________________________________________________
lstm_22 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 303       
Total params: 206,513
Trainable params: 206,513
Non-trainable params: 0
_________________________________________________________________


In [319]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [320]:
model.fit(x_train,y_train,epochs=2)

Train on 10980 samples
Epoch 1/2
10980/10980 [==============================] - 29s 3ms/sample - loss: 0.7995 - accuracy: 0.6572
Epoch 2/2
10980/10980 [==============================] - 21s 2ms/sample - loss: 0.5401 - accuracy: 0.7798


In [321]:
model.evaluate(x_train,y_train,verbose=0)

[0.39640540186933265, 0.85701275]

In [186]:
test_df=pd.read_csv('test.csv')

In [187]:
test_df.head()

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,569682010270101504,American,NaN,zsalim03,NaN,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada)
1,569608307184242688,American,NaN,sa_craig,NaN,0,"@AmericanAir after all, the plane didn’t land ...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada)
2,567879304593408001,Southwest,NaN,DanaChristos,NaN,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada)
3,569757651539660801,US Airways,NaN,rossj987,NaN,0,@USAirways I can legitimately say that I would...,NaN,2015-02-22 23:16:24 -0800,"Washington, D.C.",Eastern Time (US & Canada)
4,569900705852608513,American,NaN,tranpham18,NaN,0,@AmericanAir still no response from AA. great ...,NaN,2015-02-23 08:44:51 -0800,New York City,Eastern Time (US & Canada)


In [197]:
texts_test=test_df['text'].values

In [204]:
cleaned_texts_test=get_cleaned_texts(texts_test)

americanair car gng dfw pull ago icy road aa since ca reach arpt wat
americanair plane land identical bad condition grk accord metars
southwestair ca believe many pay customer left high dry reason flight cancel flightlations monday bdl wow
usairways legitimately say would rather driven cross country flown u airway
americanair still response aa great job guy
united developer fly tmrw morn min layover earlier flight layover move
usairways hello anyone
usairways husainhaqqani husain u shld protest well one ur party member rehman malik delayed pia flight
usairways likely flightaware say plane still durango depart
americanair even give option say line busy plz try late flightr
united announcement pre boarding address mobility disability require travel lot stuff preboard
usairways really embarrass ask complimentary detailed http amp argue
southwestair passport time trip could still fly photo id thingsishouldknow ifeeldumb
americanair delayed bag friend lisa pafe get bag day costa rica issue 

southwestair aww thanks love
usairways make thx
guy go start fly paris virginamerica lizautter welcome
fuck united shitty fuck information http
usairways americanair day amp still luggage chkd http asgmnt driver since yest
americanair mattthomasnews american airline start fly siberia wait usa oy
southwestair delay flight bdl mco need get florida amp race daytona
united nope case id either time
jetblue remember date monday night back nyc heartbroken
united people status get buy upgd much less cost ahead premier earn confirm certificate
southwestair strives centric everything communication advertising customer journey etc anamarketers
jetblue ceo battle appease passenger wall street waterbury republican american http
united hey awesome thanks reply fill form americanair
virginamerica start flight dallas dallaslovefield austin avgeek
jetblue make stay jfk late flightr let rebook without make pay
southwestair employee bwi amaze
southwestair spare ticket vega would forever grateful destinat

united service desk gate help u find flight get u dest time flight get cancel flightled
jetblue closepwcs please tweet fly jetblue every time
southwestair go court gain access delta gate love field http deltanews
americanair look like mistake
usairways like call v try since yesterday one answer
usairways thank glad home lot delay plane flight crew show frustrate
jetblue long day arrive london morning
never get strip fast stoke special imaginedragons show tonight thx southwestair http
usairways month old suffer rare form cancer customer service agent compassion disgust
virginamerica flight love field sfo definitely cancel flightled monday february
usairways friend hold hour still counting
united sent dm last week suit regard error response give please answer dm tell send suit
usairways tweet get response email dmsc
united nut say flight ground still albany wo plane someone compensate
united let know week next another rt
usairways americanair plane left agent door close early
united star

united time less important strandusindenver hourandtenminutedelay flightfail
southwestair answer right away new re ca rebook people suppose leave today bad customer service
americanair lack american air chance
usairways contact people still answer keep send expedia keep send
southwestair thanks add straight flight columbus oakland
virginamerica access website safari iphone seem work mac ipad need iphone add passbook
usairways something wrong phone system call someone play record flight notification late flight
united hey many time change ua go tonight mia ewr
united think guy half full flight held overbooked flight fill one cancel flightled mine
americanair hi name email aadvantage website wo let reset password troubleshoot together thanks
united pay person portland airport fly perth do u air
southwestair want money paid early bird refund noone seem able three different sw work
usairways even single unreasonable human planet earth excuse treatment
jetblue come
virginamerica way change 

united flight nightmare poor customer service confirm seat give away issue
usairways spoke rep nh pa poor communication bag still sit philly train staff
usairways twitter reservation connect people want reservation duh
jetblue app remind fly fll week chillycvz thankgoodness mypompanobeach family friend sunshine
lololololol jetblue fleet fleek http
americanair bos
jetblue go blue go back
southwestair yep certain flight
usairways ya need work online checkout system two bank transaction one ticket bush league yell devs
southwestair flight cancel flightled monday look cancel flight credit card
virginamerica rang wait minute book ticket vendor fix site
united flight dfw iad cancel flightled mechanical get rental car origin
usairways thanks ruiningmy vacation best could do without
americanair sure bird warmer weather soon
jetblue great experience flight sfo jfk seat service food everything top quality back soon
united try one available bogota everyone rude houston stuck hour guy
usairways cr

...

united flight attendant superben super helpful find bag left flight today excellent customer service name fit
united kind unnerve watch guy deice plane text phone whole time http
usairways need help air clt phl conf need confirm bna bumped day need home
jetblue set fly bad first date giant metal bird machine even brought snack
jetblue want say flight attendant fitz best tonight flight great guy make flight fantastic
americanair email address send issue guy screw rebooked flight wrong day please help
southwestair beautiful day seattle http
united rarely cease amaze bad hope last time fly
usairways americanair empty seat open upgrade gold member would love consistent policy
usairways heard monterey airport close due thick fog update
usairways ca wait trip thanks get
americanair thanks best stay night dallas new trial detroit via atlanta tomorrow assume cancel flightlations
jetblue last sleep cali back jfk tomorrow night look forward another amaze flight
usairways yeah know spend almost d

united airline flight ua need make ua u hold plane u
united suppose board brought plane hanger wtf know flight
jetblue flight bos rsw tomorrow need wife old sit together option check
usairways empty class chairman ca move due silly award travel rule trump common sense customer service fail
americanair follow try usairways record locator number give error code
united miss incoming time sat jetbridge driver miss connection time flight
americanair feel free email worry go anywhere damn tarmac bit
united guess flight go cancel flightled empty seat confirm important frustrate
americanair really hope departs say catering service wasnt available see next plane nthin
united deceptive marketing practice promise book flight would retain status grant neverflyunited
united system hold min tell go charge flight book problem fee customer service
southwestair double comment phl anywhere south florida go go go hello u air
americanair sent letter say would talk stewardess serve drunk drink help
united 

jetblue possible way contact fraud department mail physical letter telephone email really
americanair try ten time last night morning many time try meeting make tonight atx
united boarding plane would much faster board back front instead front back makessense efficiency travel fly
jetblue thanks good sunday
united travel fra thru iad mco tomorrow concerned abt iad weather ca change advisory issue advice
united epicfail former gate agent pia walk away quit luckily responsible pia agent save day
americanair thanks follow think fair refund ticket price two one way rental car buy
jetblue airway corporation register high short interest wall street pulse http
eliza amp cheat u americanair airtahitinui amp lovely flight back lot
usairways difficult work baggage claim answer phone
americanair really need get clue start treat people like human animal americandairlinessucks
jetblue right game set match
united thats didnt read entire tweet wife flight delayed thats hour flight delay week
american

united unbelievable hypocrisy
jetblue shock misbehavior cheap comment jet airway staff passenger delhi airport friday boarding dubai disgust
southwestair good go make boston time actually wifi right
southwestair go file formal complaint never happen please direct u right department
united amaze really
southwestair today go day feel thank support throughout come home nashville octavia
united give email address send actually screen shot
americanair number even let wait next customer rep frustrate ca talk human
usairways cstsvc phl honor ur system reality rcd change hpn extra hour phl get cle
southwestair still baltimore delta lap around u laugh ridiculous
united plz advertise wifi gon na work thanks worstflightever
americanair ca dfwairport capability spread salt runway
united lack customer centricity u astound
southwestair lose kay chapman design important show wo help locate terrible customer service
southwestair yes extra wonder go yo loose room
united receive luggage also look left s

jetblue chance one flight leave newark nj cancel flightled tomorrow
jetblue since strand syr without ride miss window opportunity right matter hour
united final pls let connection leave conf http
americanair ca get thru anyone phone
usairways update flight
jetblue real shit nigga north play around
americanair good enough need provide phone number partner investigate iata thefaaonline
southwestair luxury people flight go get spoil
americanair nice people world work admiral club gladys san juan e mi amiga
united policy behind change passenger seat assignment gate
united rebooked casleah split take different flight pilot ran hour frustrate
shannonwoodward want keep update wear amp amp win nice like
united hour luggage sfo wear clothes hour thank flythefriendlyskies
united hi im fly connect time lax enough time collect bag recheck av
americanair keep try call change reservation keep hang
united thanks reply box mine miss one add bag
southwestair travel one get miscommunicated
usairways min

In [206]:
x_test=t.texts_to_sequences(cleaned_texts_test)

In [208]:
x_test=pad_sequences(x_test,max_len,padding='pre')

In [209]:
x_test.shape

(3660, 22)

In [322]:
y_pred=model.predict(x_test)

In [323]:
y_pred.shape

(3660, 3)

In [324]:
y_predicted=[]
for y in y_pred:
    y_predicted.append(np.argmax(y))

In [325]:
y_predicted=le.inverse_transform(y_predicted)

In [326]:
np.savetxt('predictions_lstm.csv',y_predicted,fmt='%s')